# Open Weather Map API

## 1. create a dummy 'cities' DataFrame

In [1]:
import pandas as pd
!pip install PyMySQL
import pymysql
import sqlalchemy 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.2 MB/s 


In [2]:
# sample cities DataFrame for exercises

cities = pd.DataFrame(data = {'City_id' : [1],'City' : ['Berlin'], 'Country_code' : ['DE']})

In [3]:
schema="gans_database"   # name of the database you want to use here
host="127.0.0.1"        # to connect to your local server
user="root"
password="SharpGajanan9860" # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

## 2. API key

Don't forget that for OpenWeatherAPI you need an API key

In [4]:
# please use your own API key
OWM_key = 'cf36567c4cb288fd1cd39a18cbfc1b5b'

## 3. Different approaches

As we saw in the Spotify example, for some API's there could be a Python wrapper that can help you get the data in easier and simpler way.  
Here, we will see two different approaches to OpenWeather API:
1. with PyOWM - Python wrapper library
2. without the wrapper

### 3.1 Approach with a wrapper (PyOWM)

Documentation for PyOWM you can find here: https://pyowm.readthedocs.io/en/latest/

In [5]:
!pip install -U pyowm

from pyowm.owm import OWM

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.5 MB 5.0 MB/s 


If you go to https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data you can get code example to get example how to get wind speed in 3h from now:

In [6]:
owm = OWM(OWM_key)
mgr = owm.weather_manager()
one_call = mgr.one_call(lat=52.5244, lon=13.4105)

one_call.forecast_hourly[3].wind().get('speed', 0)

2.83

Problem with PyOWM is that except for few code examples, they don't provide much details about their classes or methods, so working with this wrapper would have to include a lot of testing and exploration on your own, or even looking into raw code on their GitHub.

#### *BONUS* If you still want to use a wrapper, check the following code for an example:

In [7]:
owm = OWM(OWM_key)
mgr = owm.weather_manager()

In [8]:
mgr_geo = owm.geocoding_manager()

# geocode Berlin (no country specified) - we'll get many results
list_of_locations = mgr_geo.geocode(cities['City'][0])
city = list_of_locations[0]  # taking the first Berlin in the list
lat = city.lat
lon = city.lon 

In [9]:
one_call = mgr.one_call(lat=lat, lon=lon, units='metric', timeformat='date')

hourly_forecast = one_call.forecast_hourly

In [11]:
weather_dict = {'city_id': [],
                'forecast_time': [],
                'outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'wind_speed': [],
                'pop': []}
for i in range(round(len(hourly_forecast)/3)):
  index = i*3 #because we want forecast for every 3 hours
  weather_dict['city_id'].append(cities['City_id'][0])
  weather_dict['temperature'].append(hourly_forecast[index].temperature().get('temp'))
  weather_dict['wind_speed'].append(hourly_forecast[index].wind().get('speed'))
  weather_dict['forecast_time'].append(hourly_forecast[index].reference_time(timeformat='iso'))
  weather_dict['outlook'].append(hourly_forecast[index].detailed_status)
  weather_dict['temperature_feels_like'].append(hourly_forecast[index].temperature().get('feels_like'))
  weather_dict['pop'].append(hourly_forecast[index].precipitation_probability)



In [12]:
weather_df = pd.DataFrame(weather_dict)

## 3.2 Approach without a wrapper

OpenWeatherMap API has a nice documentation and you can find it here: https://openweathermap.org/api

We are going to use 5 day / 3 hour forecast https://openweathermap.org/forecast5

Provided at their docmentation you can find an example of API call with only having info about city name and country_code
https://openweathermap.org/forecast5#name5

`api.openweathermap.org/data/2.5/forecast?q={city name},{country code}&appid={API key}`

- city name and country code have to be in format e.g. ('Berin,DE')
- appid is your unique API key
- you can also add units which can be 'standard', 'metric', 'imperial'
- or lang for languages e.g. 'de', 'it' etc.
- or cnt for number of timestamps in response

You can also find an example of API call with having info about latitude and longitude  
https://openweathermap.org/forecast5#geo5

api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API key}

You can choose your own approach. We will show here the approach with city name and country code

#### 3.2.1 Creating a request

In [13]:
import requests

In [14]:
url = f"http://api.openweathermap.org/data/2.5/forecast?q={cities['City'][0]},{cities['Country_code'][0]}&appid={OWM_key}&units=metric"
response = requests.get(url)

In [15]:
response = requests.get(url)
response.status_code

200

#### 3.2.2 Exploring the response (json)

You can copy paste your json response output to one of many online json viewers to see your json in nicer and more practical way
http://jsonviewer.stack.hu/

In [16]:
#response.json()

In [17]:
#This will only work in jupyter notebook so if you are using colab you can skip it

from IPython.display import JSON
# JSON() helps us preview the json in scalable way
JSON(response.json())
# under the node 'list' are the actual weather informations for each hour
JSON(response.json()['list'])

<IPython.core.display.JSON object>

#### 3.2.3 Gathering the data we need from json

In [18]:
weather_dict = {'city_id': [],
                'forecast_time': [],
                'outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'wind_speed': [],
                'pop': []}
# iterating over items in 'list' node and selecting the subnodes gives us the exact info we need 
for hour in response.json()['list']:
  weather_dict['city_id'].append(cities['City_id'][0])
  weather_dict['temperature'].append(hour['main']['temp'])
  weather_dict['wind_speed'].append(hour['wind']['speed'])
  weather_dict['forecast_time'].append(hour['dt_txt'])
  weather_dict['outlook'].append(hour['weather'][0]['description'])
  weather_dict['temperature_feels_like'].append(hour['main']['feels_like'])
  weather_dict['pop'].append(hour['pop'])


#### 3.2.4 Creating a new weather DataFrame

In [19]:
weather_df = pd.DataFrame(weather_dict)

,city_id,forecast_time,outlook,temperature,temperature_feels_like,wind_speed,pop
0,1,2022-10-14 12:00:00,few clouds,16.66,16.34,2.90,0.00
1,1,2022-10-14 15:00:00,scattered clouds,16.66,16.21,1.98,0.00
2,1,2022-10-14 18:00:00,light rain,14.77,14.29,2.40,0.20
3,1,2022-10-14 21:00:00,overcast clouds,12.69,12.13,2.69,0.00
4,1,2022-10-15 00:00:00,light rain,13.47,12.94,3.30,0.35
5,1,2022-10-15 03:00:00,light rain,13.24,13.07,3.21,0.96
6,1,2022-10-15 06:00:00,light rain,12.90,12.75,2.48,0.79
7,1,2022-10-15 09:00:00,broken clouds,15.52,15.19,2.88,0.00
8,1,2022-10-15 12:00:00,broken clouds,18.58,18.03,4.09,0.00
9,1,2022-10-15 15:00:00,broken clouds,18.47,17.91,4.38,0.00
